In [1]:
import pandas as pd
# import pandas.io.sql as psql
# import numpy as np
from sqlalchemy import create_engine, event
import urllib.parse
import time
# import gc

# Inputs & Outputs

In [2]:
#SQL
server = 'CSKMA0400\RDB_Data'
db = 'JLDJobPath'
odbc_connection_string = 'DRIVER={SQL Server Native Client 11.0};SERVER='+server+';DATABASE='+db+';Trusted_Connection=yes'

sql_table_0 = "linkedclaims_casuals_2018m04_v2_flat_20140101_with_income"
sql_table_1 = "linkedclaims_casuals_2018m04_v2_flat_20141001_with_income"
sql_table_2 = "linkedclaims_casuals_2018m04_v2_flat_20150401_with_income"
sql_table_3 = "linkedclaims_casuals_2018m04_v2_flat_20160101_with_income"

sql_tables_all = [sql_table_0, sql_table_1, sql_table_2, sql_table_3]

sql_table_out = "linkedclaims_casuals_2018m04_v2_flat_multi_with_income"

varprofile_csvfilename = "D:/DATA/linkedclaims_casuals_2018m04_v2_flat_20140101_with_income_with_edu_variable_profile.csv"
variables_to_add= ['ppsn','EndDate_0']

# Procedure

In [3]:
def drop_sql_table(odbc_connection_string, sql_table):
    # Connect to SQL
    params = urllib.parse.quote_plus(odbc_connection_string)
    engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
    conn = engine.connect().connection
    cursor = conn.cursor()

    # Drop table if exists
    sql_string_drop = "IF OBJECT_ID('"+ sql_table + "', 'U') IS NOT NULL" +'\n'+ "DROP TABLE " + sql_table
    cursor.execute(sql_string_drop)
    conn.commit()
    conn.close()

def copy_data_from_to_sql(odbc_connection_string, sql_table_from, sql_table_to, variables):
    # Connect to SQL
    params = urllib.parse.quote_plus(odbc_connection_string)
    engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
    conn = engine.connect().connection
    cursor = conn.cursor()
    
    sel_var= (',').join(variables)
    sql_statement = 'SELECT ' + sel_var + ' INTO '+ sql_table_to + ' FROM ' + sql_table_from
    
    cursor.execute(sql_statement)
    conn.commit()
    
    #Close SQL Connection
    conn.close()
    return

def merge_data_from_to_sql(odbc_connection_string, sql_table_from, sql_table_to, variables):
    # Connect to SQL
    params = urllib.parse.quote_plus(odbc_connection_string)
    engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
    conn = engine.connect().connection
    cursor = conn.cursor()
    
    sel_var= (',').join(variables)
    sql_statement = 'Insert into ' + sql_table_to + ' select '+ sel_var + ' from ' + sql_table_from
    
    cursor.execute(sql_statement)
    conn.commit()
    
    #Close SQL Connection
    conn.close()
    return


variables = pd.read_csv(varprofile_csvfilename)
variables = variables[variables.selected==1].Variable.tolist()
variables.extend(variables_to_add)


print ("Output table %s" %(sql_table_out))
drop_sql_table(odbc_connection_string, sql_table_out)

print ("Copy table %s" %(sql_tables_all[0]))
copy_data_from_to_sql(odbc_connection_string, sql_tables_all[0], sql_table_out, variables)

for dt in range(1,len(sql_tables_all)):
    
    print ("Merge table %s" %(sql_tables_all[dt]))
    merge_data_from_to_sql(odbc_connection_string, sql_tables_all[dt], sql_table_out, variables)
    
print ("ALL DONE")

Output table linkedclaims_casuals_2018m04_v2_flat_multi_with_income
Copy table linkedclaims_casuals_2018m04_v2_flat_20140101_with_income
SELECT End_weekly_rate_0,LM_code_rank_0,age,duration_days_0,sex,total_duration_days,total_duration_days_nat_code_Irish,total_max_LM_code_rank,total_max_ada_code_rank,total_max_family_flag_rank,total_max_marital_status_rank,total_max_occupation_rank,total_min_LM_code_rank,total_min_ada_code_rank,total_min_family_flag_rank,total_min_marital_status_rank,total_min_occupation_rank,Class_A_Earn0,Class_A_Earn1,Class_A_Earn2,Class_A_Earn3,Class_A_Earn4,Class_S_Earn0,Class_S_Earn1,Class_S_Earn2,Class_S_Earn3,Class_S_Earn4,Class_Other_Earn0,Class_Other_Earn1,Class_Other_Earn2,Class_Other_Earn3,Class_Other_Earn4,Class_A_weeks0,Class_A_weeks1,Class_A_weeks2,Class_A_weeks3,Class_A_weeks4,ppsn,EndDate_0 INTO linkedclaims_casuals_2018m04_v2_flat_multi_with_income FROM linkedclaims_casuals_2018m04_v2_flat_20140101_with_income
Merge table linkedclaims_casuals_2018m04_